# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# -----------------------------------------
# STEP 1: DATA ENGINEERING
# -----------------------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Loading the Tic Tac Toe dataset
ttt_dataset = pd.read_csv("tic-tac-toe.csv")   

display(ttt_dataset.head())
print("\nDataset shape:", ttt_dataset.shape)


display(ttt_dataset.info())
print(ttt_dataset.nunique())

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True



Dataset shape: (958, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


None

TL       3
TM       3
TR       3
ML       3
MM       3
MR       3
BL       3
BM       3
BR       3
class    2
dtype: int64


In [3]:
# -----------------------------
# Encoding Board Values (x, o, b)
# -----------------------------
mapping = {"x": 1, "o": -1, "b": 0}

for col in ttt_dataset.columns[:-1]:
    ttt_dataset[col] = ttt_dataset[col].map(mapping)

# Fix class mapping (case-sensitive)
ttt_dataset["class"] = ttt_dataset["class"].map({True: 1, False: 0})

print("Unique class labels:", ttt_dataset["class"].unique())


Unique class labels: [1 0]


In [5]:
# -----------------------------
# Splitting Inputs (X) and Output (y)
# -----------------------------

X = ttt_dataset.iloc[:, :-1].values   # board positions
y = ttt_dataset.iloc[:, -1].values    # win/lose

# -----------------------------
# Normalizing Input Data
# -----------------------------

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)


print("Preprocessing complete!")

Preprocessing complete!


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
print(ttt_dataset["class"].unique())


[1 0]


In [9]:
# -----------------------------
# Train-Test Split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

Training samples: 766
Testing samples: 192


In [11]:
# -------------------------------------------------------
# Building Sequential Neural Network (Recommended Syntax)
# -------------------------------------------------------

model = keras.Sequential([
    keras.Input(shape=(9,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(2, activation='softmax')   # 2 classes: X wins (1) or not (0)
])

# -------------------------------------------------------
# Compiling the Model
# -------------------------------------------------------

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
# -------------------------------------------------------
# Training the Model
# -------------------------------------------------------

history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)


Epoch 1/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.5197 - loss: 0.7064 - val_accuracy: 0.6494 - val_loss: 0.6536
Epoch 2/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6784 - loss: 0.6022 - val_accuracy: 0.7403 - val_loss: 0.6021
Epoch 3/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7360 - loss: 0.5277 - val_accuracy: 0.7727 - val_loss: 0.5558
Epoch 4/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7770 - loss: 0.4785 - val_accuracy: 0.7662 - val_loss: 0.5076
Epoch 5/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8210 - loss: 0.4374 - val_accuracy: 0.7792 - val_loss: 0.4744
Epoch 6/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8643 - loss: 0.3646 - val_accuracy: 0.7922 - val_loss: 0.4318
Epoch 7/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8707 - loss: 0.3362 - val_accuracy: 0.8571 - val_loss: 0.3637
Epoch 8/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8975 - loss: 0.2952 - val_accuracy: 0.8766 - val_l

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [15]:
# -------------------------------------------------------
# Evaluating the Model
# -------------------------------------------------------

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", accuracy)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9850 - loss: 0.0469
Test Accuracy: 0.9739583134651184


In [19]:
# -------------------------------------------------------
# Saving the Model
# -------------------------------------------------------

model.export("tic-tac-toe.model")
print("Model saved successfully as a SavedModel folder.")


INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


Saved artifact at 'tic-tac-toe.model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2682289549584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289550352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289548240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289551504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289551312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289550160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289551120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2682289551888: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved successfully as a SavedModel folder.


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

### ⭐ Interpretation

#### Training loss = 0.0015 → extremely low (< 0.1 ✔)

#### Training accuracy = 1.00 → perfect (> 0.95 ✔)

#### Validation accuracy = 0.9935 → extremely good (> 0.95 ✔)

#### Validation loss = 0.0261 → very low (< 0.1 ✔)

### 🎉 Conclusion

#### ➡️ The model ALREADY meets and exceeds the performance requirements.
#### ➡️ No further improvements are required, because both training and validation metrics are extremely strong and show no signs of underfitting or significant overfitting.

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here